<a href="https://colab.research.google.com/github/random-object/ai/blob/main/Zags_Main_3_7fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Stopped mid level = 2? use continue
# Stopped mid level = 1 or 0? use upsample

lemode = 'primed'     # 'ancestral','primed','continue','cutcontinue','upsample'
lemodel = '5b_lyrics'                          #5b_lyrics or '5b' or '1b_lyrics'

lecount = 3
lesample_length_in_seconds = 90
lesampling_temperature = .98
lehop = [.75,1,.125]                 #default [.5,.5,.125], optimized [1,1,0.0625]

lepath = '/content/gdrive/MyDrive/samplessongai'

leprompt_length_in_seconds=12  
leaudio_file = '/content/gdrive/MyDrive/magicshadow.wav'                    

lecut = 70               # used only on cutcontinue
transpose = [0,1,2]      # used only on cutcontinue [0,1,2] = default, ex [1,1,1] all samples are copied from item 1

leexportlyrics = False
leprogress = True
leautorename = True

leartist = "burzum"
legenre = "black"
lelyrics = """When the Black clouds cover the moon
And reality turns into nightmare
Lighting cuts through the dark sky
Almighty Perun gave us his sign !
Brothers, wolves, I summon you !
Wind will bring you my howling
And smell of the human blood
Yes, I found another sacrifice
Droops of blood upon the stone
I lick the cold virgins corpse
I an blinded by my bloodlust
I the Northern Carpathians
Mystic eyes between the trees
Brothers hungry for the blood
Through the blood I find desire
The only sense of the evil life
Immortality is true
For the creatures without shadows
There is hell on the earth
In the Northern Carpathians...
Unholy woods of mysteries
Alone witches full of hate
Werewolves hidden in the caves
Forgotten spells of pagan past
Long nights and hours of darkness
Cold mountain winds blow through the trees
Diabolic feast around the fire
Dark shadows dancing with flames
There is a place where Evil lives
There are woods where people die
At midnight blood is more precious than gold
In the Northern Carpathians...
"""

lechunk_size = 16 
lemax_batch_size = lecount
lelower_batch_size = lechunk_size
lelower_level_chunk_size = lechunk_size * 2

# 1 Sample


In [ ]:
if lemode=='ancestral':
  leprompt_length_in_seconds=None  
  leaudio_file = None
###############################################################################
###############################################################################

codes_file=None

!pip install git+https://github.com/openai/jukebox.git

##$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$#### autosave start
import os
from glob import glob

filex = "/usr/local/lib/python3.7/dist-packages/jukebox/sample.py"
fin = open(filex, "rt")
data = fin.read()
fin.close()

newtext = '''import fire
import os
from glob import glob

from termcolor import colored
from datetime import datetime

newtosample = True'''
data = data.replace('import fire',newtext)

newtext = '''starts = get_starts(total_length, prior.n_ctx, hop_length)
        counterr = 0
        x = None
        for start in starts:'''
data = data.replace('for start in get_starts(total_length, prior.n_ctx, hop_length):',newtext)

newtext = '''global newtosample
    newtosample = (new_tokens > 0)
    if new_tokens <= 0:'''
data = data.replace('if new_tokens <= 0:',newtext)

newtext = '''counterr += 1
            datea = datetime.now()		
            zs = sample_single_window(zs, labels, sampling_kwargs, level, prior, start, hps)			
            if newtosample and counterr < len(starts):
                del x; x = None; prior.cpu(); empty_cache()
                x = prior.decode(zs[level:], start_level=level, bs_chunks=zs[level].shape[0])
                logdir = f"{hps.name}/level_{level}"
                if not os.path.exists(logdir):
                    os.makedirs(logdir)
                t.save(dict(zs=zs, labels=labels, sampling_kwargs=sampling_kwargs, x=x), f"{logdir}/data.pth.tar")
                save_wav(logdir, x, hps.sr)
                del x; prior.cuda(); empty_cache(); x = None
            dateb = datetime.now()
            timex = ((dateb-datea).total_seconds()/60.0)*(len(starts)-counterr)
            print(f"Step " + colored(counterr,'blue') + "/" + colored( len(starts),'red') + " ~ New to Sample: " + str(newtosample) + " ~ estimated remaining minutes: " + (colored('???','yellow'), colored(timex,'magenta'))[counterr > 1 and newtosample])'''
data = data.replace('zs = sample_single_window(zs, labels, sampling_kwargs, level, prior, start, hps)',newtext)


newtext = """lepath=hps.name
        if level==2:
          for filex in glob(os.path.join(lepath + '/level_2','item_*.wav')):
            os.rename(filex,filex.replace('item_',lepath.split('/')[-1] + '-'))
        if level==1:
          for filex in glob(os.path.join(lepath + '/level_1','item_*.wav')):
            os.rename(filex,filex.replace('item_',lepath.split('/')[-1] + '-L1-'))
        if level==0:
          for filex in glob(os.path.join(lepath + '/level_0','item_*.wav')):
            os.rename(filex,filex.replace('item_',lepath.split('/')[-1] + '-L0-'))
        save_html("""
if leautorename:
  data = data.replace('save_html(',newtext)

if leexportlyrics == False:
  data = data.replace('if alignments is None','#if alignments is None')
  data = data.replace('alignments = get_alignment','#alignments = get_alignment')
  data = data.replace('save_html(','#save_html(')

if leprogress == False:
  data = data.replace('print(f"Step " +','#print(f"Step " +')
  
fin = open(filex, "wt")
fin.write(data)
fin.close()
##$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$#### autosave end

import jukebox
import torch as t
import librosa
import os

from datetime import datetime

from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample, \
                           load_prompts
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
rank, local_rank, device = setup_dist_from_mpi()

print(datetime.now().strftime("%H:%M:%S"))

model = lemodel
hps = Hyperparams()
hps.sr = 44100
hps.n_samples = lecount 
hps.name = lepath

chunk_size = lechunk_size
max_batch_size = lemax_batch_size

hps.levels = 3
hps.hop_fraction = lehop

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

# Prime song creation using an arbitrary audio sample.
mode = lemode
codes_file=None
audio_file = leaudio_file
prompt_length_in_seconds=leprompt_length_in_seconds


if os.path.exists(hps.name):
  # Identify the lowest level generated and continue from there.
  for level in [0, 1, 2]:
    data = f"{hps.name}/level_{level}/data.pth.tar"
    if os.path.isfile(data):
      mode = mode if 'continue' in mode else 'upsample'
      codes_file = data
      print(mode + 'ing from level ' + str(level))
      break
print('mode is now '+mode)

sample_hps = Hyperparams(dict(mode=mode, codes_file=codes_file, audio_file=audio_file, prompt_length_in_seconds=prompt_length_in_seconds))

sample_length_in_seconds = lesample_length_in_seconds 
hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

metas = [dict(artist = leartist,
            genre = legenre,
            total_length = hps.sample_length,
            offset = 0,
            lyrics = lelyrics,
            ),
          ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]


#----------------------------------------------------------2


sampling_temperature = lesampling_temperature
lower_batch_size = lelower_batch_size
max_batch_size = lemax_batch_size
lower_level_chunk_size = lelower_level_chunk_size
chunk_size = lechunk_size 
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True, 
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

if sample_hps.mode == 'ancestral':
  zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cuda') for _ in range(len(priors))]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'upsample':
  assert sample_hps.codes_file is not None
  # Load codes.
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cuda() for z in data['zs']]
  assert zs[-1].shape[0] == hps.n_samples, f"Expected bs = {hps.n_samples}, got {zs[-1].shape[0]}"
  del data
  print('Falling through to the upsample step later in the notebook.')
elif sample_hps.mode == 'primed':
  assert sample_hps.audio_file is not None
  audio_files = sample_hps.audio_file.split(',')
  duration = (int(sample_hps.prompt_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  x = load_prompts(audio_files, duration, hps)
  zs = top_prior.encode(x, start_level=0, end_level=len(priors), bs_chunks=x.shape[0])
  print(sample_hps.prompt_length_in_seconds)
  print(hps.sr)
  print(top_prior.raw_to_tokens)
  print('aaaaaaaaaaaaaaaaaaaaaaaaaaaa 4.52')
  print(duration)
  print(audio_files)
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'continue':
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cuda() for z in data['zs']]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'cutcontinue':
  print('-------CUT INIT--------')
  lecutlen = (int(lecut*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  print(lecutlen)
  data = t.load(codes_file, map_location='cpu')
  zabaca = [z.cuda() for z in data['zs']]
  print(zabaca)
  assert zabaca[-1].shape[0] == hps.n_samples, f"Expected bs = {hps.n_samples}, got {zs[-1].shape[0]}"
  priorsz = [top_prior] * 3
  top_raw_to_tokens = priorsz[-1].raw_to_tokens
  assert lecutlen % top_raw_to_tokens == 0, f"Cut-off duration {lecutlen} not an exact multiple of top_raw_to_tokens"
  assert lecutlen//top_raw_to_tokens <= zabaca[-1].shape[1], f"Cut-off tokens {lecutlen//priorsz[-1].raw_to_tokens} longer than tokens {zs[-1].shape[1]} in saved codes"
  zabaca = [z[:,:lecutlen//prior.raw_to_tokens] for z, prior in zip(zabaca, priorsz)]
  hps.sample_length = lecutlen
  print(zabaca)
  zs = _sample(zabaca, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
  del data
  print('-------CUT OK--------')
  hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zibica = [z.cuda() for z in data['zs']]
  zubu = zibica[:]
  if transpose != [0,1,2]:
    zubu[2][0] = zibica[:][2][transpose[0]];zubu[2][1] = zibica[:][2][transpose[1]];zubu[2][2] = zibica[:][2][transpose[2]]
    zubu[1][0] = zibica[:][1][transpose[0]];zubu[1][1] = zibica[:][1][transpose[1]];zubu[1][2] = zibica[:][1][transpose[2]]
    zubu[0][0] = zibica[:][0][transpose[0]];zubu[0][1] = zibica[:][0][transpose[1]];zubu[0][2] = zibica[:][0][transpose[2]]
  zubu = _sample(zubu, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
  print('-------CONTINUE AFTER CUT OK--------')
  zs = zubu
else:
  raise ValueError(f'Unknown sample mode {sample_hps.mode}.')



print(datetime.now().strftime("%H:%M:%S"))



  Cloning https://github.com/openai/jukebox.git to /tmp/pip-req-build-yr7tajjq
  Running command git clone -q https://github.com/openai/jukebox.git /tmp/pip-req-build-yr7tajjq
     |████████████████████████████████| 60 kB 5.9 MB/s 
     |████████████████████████████████| 238 kB 10.8 MB/s 
     |████████████████████████████████| 3.5 MB 51.3 MB/s 
     |████████████████████████████████| 1.6 MB 14.0 MB/s 
     |████████████████████████████████| 2.4 MB 48.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 20.2 MB 104 kB/s 
  Created wheel for jukebox: filename=jukebox-1.0-py3-none-any.whl size=197917 sha256=786b49b03213719e17343503724e6af2ff3d9374f8064db881e143b1d727bb1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-_49nffbz/wheels/d6/42/39/91f8a32505a445499702ae0f887769e6bb5030c42382d74ae0
  Created wheel for fire: filename=fire-0.1.3-py2.py3-none-any.whl size=497

# 2 Upsample


In [ ]:
print(datetime.now().strftime("%H:%M:%S"))
del top_prior
empty_cache()
top_prior=None

upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)
print(datetime.now().strftime("%H:%M:%S"))

